In [10]:
# This is an example snippet!
# To create your own, add a new snippet block to the
# snippets.json file in your jupyter nbextensions directory:
# /nbextensions/snippets/snippets.json
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [1]:
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import requests 

from IPython.core.display import clear_output
from time import time
start_time = time()
# from warnings import warnwarn("Warning Simulation")

#from selenium import webdriver
#from selenium.webdriver.chrome.options import Options
#import pandas as pd

# def get_content_using_Selenium(url):
#     DRIVER_PATH = 'C:\Users\Nguyen Gia Bao\Desktop\chromedriver_win32\chromedriver'

#     options = Options()
#     options.headless = True
#     options.add_argument("--window-size=1920,1200")

#     driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
#     driver.get(url)
#     htmlContent = BeautifulSoup(driver.page_source)
#     driver.quit()
#     return htmlContent

def request_page(url):
    headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
    r = requests.get(url, headers = headers) 
    soup = BeautifulSoup(r.content,'html5lib')
    return soup

# Get list of post
def get_list_of_posts(table, attrs1, attrs2):
    listRental = []
    for row in table.findAll('div', attrs = {'class': attrs1}): 
        quote = {}
        row1 = row.find('div', attrs = {'class': attrs2})
        listRental.append(row1.a['href'])
    return listRental

# Get detail info for each post: Đặc điểm BĐS + Liên hệ
def get_detail_info(table_content,attrs1, attrs2):
    if not table_content:
        print('table_content is empty')
    else:
        key_table = table_content.findAll('div',attrs = {'class':attrs1})
        value_table = table_content.findAll('div',attrs = {'class':attrs2})
        x = {k.text.strip():v.text.strip() for k, v in zip(key_table, value_table)}
        #d = {k: v for k, v in zip(name_list, num_list)
        return x

# Get link den tung post tren tung page
def create_individual_link(url_base, listPosts ):
    listUrl=[]
    for i in listPosts:
        listUrl.append(str(url_base + i))    
    return listUrl

# Get more info: Ma tin dang, loai tin, ngay dang, ngay het han
def get_more_info(tableMoreInfo):
    divList = tableMoreInfo.find_all('div')
    d = {}
    for i in range(len(divList)):
        if i == 1:
            pass
        elif i == 2:
            k = divList[i].span.string
            v = divList[i].find('span', id = 'ltrVipType').text
        else: 
            k,v = divList[i].text.replace('\n','').strip().split(':')
        d[k] = v  
    return d

def print_list(List):
    for i in List:
        print(i)
        
def error_empty(table):
    if not table:
        print('table is empty')

def create_dataFrame(listData):
    data = []
    for i in listData:
        data.append(i[0])
    df = pd.DataFrame(data, columns=['Mã tin đăng',
                                     'Loại hình tin đăng:',
                                     'Ngày đăng',
                                     'Ngày hết hạn',
                                     'Tên liên lạc',
                                     'Điện thoại',
                                     'Mobile',
                                     'Email',
                                     'Loại tin rao',
                                     'Địa chỉ',
                                     'Số phòng ngủ',
                                     'Số toilet',
                                     'Nội thất',
                                     'Hướng nhà',
                                     'Hướng ban công',
                                     'Content'
                                     ])
    return df

#============ NHA DAT CHO THUE ======================================================================
# def get_content_NhaDatChoThue(soup, attrsContent, attrsPropertyDetail, attrsProject, attrsContactDetail, attrsProjectDetail, attrsMoreInfo):
def get_content_NhaDatChoThue(soup):
    listData=[]
    d = {}
    
    # get more info
    attrsMoreInfo = 'prd-more-info'
    tableMoreInfo = soup.find('div', attrs = {'class':attrsMoreInfo}) 
    if not tableMoreInfo:
        print('tableMoreInfo is empty')
    else:
        moreInfo = get_more_info(tableMoreInfo)
        d.update(moreInfo)
    
    # get Contact Detail
    attrsContactDetail ='divCustomerInfo'
    tableContactDetail = soup.find('div', attrs = {'id':attrsContactDetail})
    if not tableContactDetail:
        print('tableContactDetail is empty')
    else:
        contactDetail = get_detail_info(tableContactDetail,'left','right')
        contactDetail['Email'] = ''
        # contactDetail['Email'] = tableContactDetail.find('div', id = 'contactEmail').a.text
        # contactDetail['Mobile'] = tableContactDetail.find('div', attrs = {'class':'right contact-phone'}).span['raw'].strip()
        d.update(contactDetail)
        # print(contactDetail)
        
    # get Property Detail
    attrsPropertyDetail ='product-other-detail'
    tablePropertyDetail = soup.find('div', attrs = {'id':attrsPropertyDetail})
    if not tablePropertyDetail:
        print('tablePropertyDetail is empty')
    else:
        propertyDetail = get_detail_info(tablePropertyDetail,'left','right')
        d.update(propertyDetail)
        # print(propertyDetail)

    
    # get Project Detail
    attrsProject ='project'
    attrsProjectDetail = 'table-detail'
    tableProject = soup.find('div', attrs = {'id':attrsProject})
    if not tableProject:
        print('tableProject is empty')
    else:
        tableProjectDetail = tableProject.find('div', attrs = {'class':attrsProjectDetail}) 
        projectDetail = get_detail_info(tableProjectDetail,'left','right')
        d.update(propertyDetail)
        # print(projectDetail)

    # get Content
    attrsContent = 'pm-content'
    tableContent = soup.find('div', attrs ={'class': attrsContent})
    if not tableContent:
        print('tableContent is empty')
    else:
        d['Content'] = tableContent.text.strip().replace('\n','')
        # print(tableContent.text.strip().replace('\n',''))
    
    listData.append(d)
    return listData


In [2]:
#==================== RUN ==============================
#******* Nha dat cho thue *************************
url_base = 'https://batdongsan.com.vn'
u1_attrs1 = 'search-productItem'
u1_attrs2 = 'p-title'
listData = []
listUrl = []

for i in range(0,100):
    # loop through all the page
    url = str(url_base + '/nha-dat-cho-thue/p' + str(i))
    
    print('===================== Page '+ str(i) +' ===================================================================')
    print(url)
    
    #Step 1: get page content
    s = request_page(url)
    #print(s)
    # s = get_content_using_Selenium(u1)
    sleep(randint(8,16))

    #Step 2: get list of posts on each page
    listPost = get_list_of_posts(s, u1_attrs1, u1_attrs2)
    

    #Step 3: create a list of individual link to each post
    listIndividualLink = create_individual_link(url_base, listPost)

    # get list of individual link for each post in each page
    dic = {}
    dic['page'+ str(i)] = listIndividualLink
    listUrl.append(dic)

    #Step 4: get content
    for i in listIndividualLink:
        print(i)

        # loop through each page and get detail
        #soup = get_content_using_Selenium(listUrl) 
        soup = request_page(i)

        # Pause the loop
        sleep(randint(3,5))

        #listData.append(get_content_NhaDatChoThue(soup, attrsContent, attrsPropertyDetail, attrsProject, attrsProjectDetail, attrsContactDetail, attrsMoreInfo))
        listData.append(get_content_NhaDatChoThue(soup))
              
              

===================== Page 0 ===================================================================
https://batdongsan.com.vn/nha-dat-cho-thue/p0
===================== Page 1 ===================================================================
https://batdongsan.com.vn/nha-dat-cho-thue/p1
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-mai-chi-tho-phuong-an-phu-prj-the-sun-avenue/gap-2pn-day-du-tien-ich-11-5-trieu-0932-729-885-pr22051014
https://batdongsan.com.vn/cho-thue-cua-hang-ki-ot-duong-le-van-luong-phuong-nhan-chinh-prj-times-tower-hacc1-complex-building/chu-san-thuong-mai-tang-1-toa-nha-35-thanh-xuan-hn-pr25392437
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-dien-bien-phu-phuong-22-prj-vinhomes-central-park/nhanh-1-phong-ngu-noi-that-leasing-cetral-chi-15tr-lh-0909060957-pr25346183
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-pho-ton-duc-thang-phuong-ben-nghe-prj-vinhomes-golden-river-ba-son/chuyen-1-pn-15tr-2pn-18-tr-3pn-28-tr-4pn-pen-gia-tot-lh-09090609

https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-thuy-loi-phuong-phuoc-long-a-prj-him-lam-phu-an/6tr-th-bao-pql-70m2-2pn-wc-co-nt-view-ldmark-song-sg-0386193995-pr25926719
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-xo-viet-nghe-tinh-phuong-25-prj-green-field/ban-ch-sai-gon-gateway-2pn-tu-1-790-ty-3pn-tu-2-65-ty-bao-so-nhan-nha-ngay-0932100172-pr25393861
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-thuy-loi-phuong-phuoc-long-a-prj-him-lam-phu-an/gi-dinh-em-lm-6tr-ntcb-9tr-full-noi-tht-con-thuong-luong-0939720039-pr25926363
tableMoreInfo is empty
tableContactDetail is empty
tablePropertyDetail is empty
tableProject is empty
tableContent is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-go-dua-phuong-tam-binh-prj-sai-gon-avenue/2pn-gia-chi-6tr-3pn-gia-9tr-goi-ngay-0932100172-pr25556845
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-ben-van-don-phuong-6-prj-masteri-millennium/officel-1pn-2pn-3pn-gia-13-trieu-20-trieu-thang-full-no

===================== Page 9 ===================================================================
https://batdongsan.com.vn/nha-dat-cho-thue/p9
https://batdongsan.com.vn/cho-thue-nha-mat-pho-duong-nguyen-van-troi-phuong-mo-lao-prj-lang-viet-kieu-chau-au-euroland/chinh-chu-can-lien-ke-tai-233-ha-dong-au-pr25717494
https://batdongsan.com.vn/cho-thue-van-phong-duong-le-van-luong-phuong-nhan-chinh-prj-diamond-flower-tower/tron-goi-tai-uu-dai-dac-biet-trong-thang-01-2019-pr18288713
https://batdongsan.com.vn/cho-thue-nha-rieng-duong-nguyen-van-tao-76/can-can-be-gan-truong-quoc-te-ba-i-pr25987741
tableProject is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-ben-van-don-phuong-12-prj-icon-56/56-1-png-ngu-view-song-50m2-18tr-free-gym-boi-lh-0906-972-055-pr24265752
https://batdongsan.com.vn/cho-thue-nha-mat-pho-duong-pham-ngu-lao-phuong-ben-nghe/can-sang-gap-nguyen-can-2-tien-nguyen-trai-quan-1-gia-tot-lien-he-0347746218-p-ha-pr25844504
tableProject is empty
https://batdongsan.co

https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-ben-van-don-phuong-6-prj-river-gate/2-pn-rivergate-gia-re-16-tr-ot-9-tr-full-nt-lh-0916-020-270-dung-nhan-ky-gui-pr25966615
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-xuan-dinh-phuong-xuan-tao-prj-khu-ngoai-giao-doan/chinh-chu-cc-ngd-full-noi-that-2pn-75m2-va-3pn-140m2-gia-yeu-thuong-0973311831-pr25977863
https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-de-tham-phuong-cau-ong-lanh/giam-gia-6-thang-sieu-mini-sang-chanh-q1-lh-0982461659-pr25873078
tableProject is empty
https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-hoang-hoa-tham-phuong-ngoc-ha/chinh-chu-can-ho-mini-moi-xay-gan-lang-bac-pr18033834
tableProject is empty
https://batdongsan.com.vn/cho-thue-kho-nha-xuong-dat-duong-dang-cong-binh-75/hang-view-song-rong-370-met-pr24797059
tableProject is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-nguyen-van-huyen-phuong-xuan-la-prj-6th-element/gia-dinh-toi-6-1pn-3pn-nbcdt-den-full-g

tableProject is empty
https://batdongsan.com.vn/cho-thue-nha-rieng-duong-ngo-tat-to-phuong-22/tro-40m2-p22-quan-binh-thanh-lh-0946968199-pr12624228
tableProject is empty
https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-luy-ban-bich-phuong-tan-thanh/cao-cap-dang-can-ho-toa-622-quan-phu-pr25781257
tableProject is empty
https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-tan-trieu-xa-tan-trieu/chinh-chu-ccmini-dien-tich-18m-20m2-gia-2-8-den-3-2-lh-a-duc-0915106805-pr25568189
tableProject is empty
https://batdongsan.com.vn/cho-thue-nha-mat-pho-pho-do-nhuan-phuong-xuan-dinh/lien-ke-khu-ngoai-giao-doan-pr25723191
tableMoreInfo is empty
tableContactDetail is empty
tablePropertyDetail is empty
tableProject is empty
tableContent is empty
https://batdongsan.com.vn/cho-thue-van-phong-duong-nguyen-thi-thap-phuong-tan-quy/210-000vnd-m2-thang-toa-nha-green-country-q7-gan-lottte-sunrise-city-pr19327483
tableProject is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-ngu

...

tableProject is empty
https://batdongsan.com.vn/cho-thue-van-phong-duong-dai-lo-thang-long-phuong-my-dinh-1/tai-so-9-ng-dien-tich-70m2-pr25661131
tableProject is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-phuong-phu-do/phong-tro-dep-tuyet-voi-pr25527340
tableProject is empty
===================== Page 89 ===================================================================
https://batdongsan.com.vn/nha-dat-cho-thue/p89
https://batdongsan.com.vn/cho-thue-van-phong-duong-dai-lo-thang-long-phuong-my-dinh-1/tai-so-9-ng-dien-tich-70m2-pr25661131
tableProject is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-phuong-phu-do/phong-tro-dep-tuyet-voi-pr25527340
tableProject is empty
https://batdongsan.com.vn/cho-thue-nha-mat-pho-duong-nguyen-trai-phuong-2-15/tien-q5-dt-4x25m-3-lau-gia-50-trieu-th-pr25964863
tableProject is empty
https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-hoang-dieu-phuong-6/cao-cap-ngay-mat-tien-q4-pr15251724
tableProject is empty
https://bat

https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-d1-phuong-tan-hung-14-prj-can-ho-florita-duc-khai/ch-gia-full-noi-that-tot-nhat-thi-truong-noi-that-co-11-5tr-nhieu-0902416899-pr15666501
https://batdongsan.com.vn/cho-thue-cua-hang-ki-ot-duong-hoang-quoc-viet-7/chinh-chu-can-ngo-210-viet-pr25657478
tableProject is empty
https://batdongsan.com.vn/cho-thue-kho-nha-xuong-dat-duong-so-5-xa-phu-chan/kcn-vsip-bac-ninh-chuyen-hang-va-dich-vu-logistic-sach-dep-tieu-chuan-pr19606437
tableProject is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-nguyen-huu-tho-xa-phuoc-kieng-prj-sunrise-riverside/chinh-chu-3-png-ngu-full-noi-that-gia-10-tr-lh-0925-234-567-pr25961401
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-tan-chanh-hiep-21-phuong-tan-chanh-hiep-1-prj-hung-ngan-garden/2pn-69m2-khu-day-du-tien-ich-lh-0906539693-pr22469617
https://batdongsan.com.vn/cho-thue-nha-mat-pho-duong-tran-phu-phuong-4-13/sieu-to-1-tret-4-lau-san-thuong-dt-9x33m-co-san-dau-4-xe-hoi-gia

tableProject is empty
https://batdongsan.com.vn/cho-thue-nha-rieng-duong-nguyen-van-thu-phuong-da-kao/can-goc-1e-dakao-quan-1-dt-7x20-trong-suot-pr25956833
tableProject is empty
https://batdongsan.com.vn/cho-thue-nha-rieng-duong-pham-dang-giang-phuong-binh-hung-hoa/nguyen-can-quan-tan-pr25830941
tableProject is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-duong-quang-ham-phuong-15-6-prj-osimi-tower/3pn-full-noi-that-gia-chi-12-5-trieu-0911-921-606-pr25670982
https://batdongsan.com.vn/cho-thue-nha-mat-pho-duong-nguyen-tat-thanh-phuong-thuan-phuoc-prj-the-sunrise-bay/day-du-noi-that-3-tang-khu-do-thi-da-hai-chau-da-nang-gia-15-17tr-thang-pr25851421
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi-phuong-thao-dien-prj-masteri-an-phu/gio-hg-da-dg-gia-tot-cap-nhat-moi-ngay-pr25357292
https://batdongsan.com.vn/cho-thue-van-phong-pho-ba-trieu-phuong-nguyen-du-prj-v-building/100m2-cao-cap-tai-hai-trung-hn-pr25634049
https://batdongsan.com.vn/cho-thue-nha-

https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-tran-duy-hung-phuong-trung-hoa-4-prj-vinhomes-dcapitale/officetel-vinmes-d-capital-cau-giay-ha-noi-dien-tich-60m2-pr25955851
https://batdongsan.com.vn/cho-thue-van-phong-duong-dien-bien-phu-phuong-da-kao/77-ong-dakao-quan-1-0933-256-881-pr23131989
tableProject is empty
https://batdongsan.com.vn/cho-thue-nha-rieng-duong-khuong-trung-phuong-khuong-trung/chinh-chu-sieu-dep-cuc-thoang-mat-trung-pr25845841
tableProject is empty
https://batdongsan.com.vn/cho-thue-van-phong-pho-le-trong-tan-phuong-duong-noi-prj-khu-do-thi-geleximco-le-trong-tan/nha-dien-tich-120m-ha-dong-pr24335204
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-duong-van-be-phuong-vinh-tuy-prj-sunshine-garden/1pn-re-dep-nhat-gia-10tr-thang-mr-cuong-0976044111-pr25956737
https://batdongsan.com.vn/cho-thue-van-phong-pho-le-trong-tan-phuong-khuong-mai/g-4-mat-duong-dien-tich-108m2-thiet-ke-hop-ly-pr25956346
tableProject is empty
https://batdongsan.com.vn/cho-thue

tableProject is empty
https://batdongsan.com.vn/cho-thue-nha-mat-pho-pho-pho-duc-chinh-phuong-truc-bach/chu-142-gia-chi-6-5tr-lh-0968819699-pr25953992
tableProject is empty
https://batdongsan.com.vn/cho-thue-nha-rieng-duong-so-4-phuong-truong-tho-2/cap-4-thu-duc-gia-5-5tr-lh-0918-847-999-pr24301888
tableProject is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-ben-van-don-phuong-6-prj-river-gate/officetel-rivergate-26-38m2-full-noi-that-dep-chi-tu-8tr-10tr-th-lh-0934061086-trung-pr25953990
https://batdongsan.com.vn/cho-thue-kho-nha-xuong-dat-duong-quoc-lo-1a-72/mat-tien-tai-binh-chanh-tphcm-va-kcn-long-hau-long-an-dien-tich-da-dang-pr25530883
tableProject is empty
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-dinh-cong-phuong-dinh-cong-prj-sky-central/176-gia-chi-tu-6-5tr-lien-he-0868050550-pr23584998
https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-phan-van-dang-phuong-thanh-my-loi-prj-vista-verde/den-luc-phai-bung-chay-voi-gio-hang-re-hon-bao-gio-he

In [ ]:
data100 = listData


In [ ]:
a = create_dataFrame(data100)

In [ ]:
import pandas as pd


In [ ]:
a.to_csv("data0_100-ndt+.csv", sep=',', encoding='utf-8')

In [4]:
selenium = []
selenium = selenium.append(get_content_using_Selenium("https://batdongsan.com.vn/nha-dat-ban"))
for title in selenium.find_elements_by_id("divCustomerInfo"):
    print(title)

NameError: name 'get_content_using_Selenium' is not defined

# SELENIUM

In [7]:
def get_content_using_Selenium(url):
    DRIVER_PATH = 'C:\\Users\\Nguyen Gia Bao\\Desktop\\chromedriver_win32\\chromedriver'

#     options = Options()
#     options.headless = True
#     options.add_argument("--window-size=1920,1200")

    driver = webdriver.Chrome(executable_path=DRIVER_PATH)
    driver.get(url)
    htmlContent = BeautifulSoup(driver.page_source)
    driver.quit()
    return htmlContent

u = 'https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-pho-ton-duc-thang-phuong-ben-nghe-prj-vinhomes-golden-river-ba-son/1pn-2pn-3pn-4pn-view-g-noi-that-chau-au-o-ngay-0977771919-pr25699704'

In [ ]:
a = get_content_using_Selenium(u)
print(a)

In [6]:
from selenium import webdriver 


In [4]:
%ls


 Volume in drive C is Acer
 Volume Serial Number is 4067-7529

 Directory of C:\Users\Nguyen Gia Bao\Documents\Jupyter test

05/06/2020  03:42 pm    <DIR>          .
05/06/2020  03:42 pm    <DIR>          ..
04/06/2020  03:17 pm    <DIR>          .ipynb_checkpoints
05/06/2020  01:04 pm    <DIR>          300 ban nha
04/06/2020  08:36 pm    <DIR>          300 origin
05/06/2020  11:31 am           132,794 bs4+.ipynb
05/06/2020  03:42 pm           150,112 bs4++.ipynb
29/05/2020  03:35 pm           201,553 Combination Pandas + bs4.ipynb
05/06/2020  07:08 am           118,942 data0_50.csv
04/06/2020  08:38 pm           175,558 data0_50.xlsx
01/06/2020  06:12 pm         2,469,894 data1_100.csv
05/06/2020  07:08 am           118,942 data50_100.csv
05/06/2020  07:10 am           181,926 data50_100.xlsx
05/06/2020  02:47 pm               587 debug.log
02/06/2020  04:20 pm    <DIR>          Old data
28/05/2020  02:44 pm             2,385 pandas.csv
29/05/2020  03:31 pm             2,385 pandas2.c